In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import atecml.data
from tqdm import tqdm

In [2]:
train_df = pd.read_pickle('./01_train.dat')
#drop unknown
train_df = train_df[train_df.label!=-1].reset_index(drop=True)


predictors = [x for x in train_df.columns if x not in atecml.data.NOT_FEATURE_COLUMNS]
DateFold={}

DateFold[0] = set(atecml.data.filter_date(train_df,start_date='2017-09-05',end_date='2017-09-12').index)
DateFold[1] = set(atecml.data.filter_date(train_df,start_date='2017-09-13',end_date='2017-09-20').index)
DateFold[2] = set(atecml.data.filter_date(train_df,start_date='2017-09-21',end_date='2017-09-28').index)
DateFold[3] = set(atecml.data.filter_date(train_df,start_date='2017-09-29',end_date='2017-10-06').index)
DateFold[4] = set(atecml.data.filter_date(train_df,start_date='2017-10-07',end_date='2017-10-14').index)
DateFold[5] = list(atecml.data.filter_date(train_df,start_date='2017-10-15',end_date='2017-11-24').index)

all_list = set(train_df.index) - set(DateFold[5])
len(all_list),len(DateFold[5])

(631141, 358865)

In [3]:
idx = 1
Train_DataSet = train_df[train_df.index.isin(list(all_list - DateFold[idx]))].reset_index(drop=True)
Normal_DF = Train_DataSet[Train_DataSet['label']==0]
Fraud_DF = Train_DataSet[Train_DataSet['label']==1]

number_record_fraud = len(Fraud_DF)
number_record_normal = len(Normal_DF)
#undersample
random_normal_indices = np.array(np.random.choice(Normal_DF.index,number_record_fraud,replace=False))
filter_list = list(random_normal_indices) + list(Fraud_DF.index)
under_sample_train = Train_DataSet[Train_DataSet.index.isin(filter_list)].reset_index(drop=True)


Val_DataSet = train_df[train_df.index.isin(DateFold[5])].reset_index(drop=True)

In [4]:
target = 'Fraud'
X_train = under_sample_train[predictors]
y_train = under_sample_train[target]

X_val = Val_DataSet[predictors]
y_val = Val_DataSet[target]

In [13]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

print('设置参数')
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          }

### 交叉验证(调参)
print('交叉验证')
min_merror = float('Inf')
best_params = {}

# 准确率
print("调参1：提高准确率")
for num_leaves in range(50,400,10):
    params['num_leaves'] = num_leaves
    params['max_depth'] = -1
      
    cv_results = lgb.cv(
                          params,
                          lgb_train,
                          seed=2018,
                          nfold=3,
                          metrics=['auc'],
                          num_boost_round=500,
                          early_stopping_rounds=50,
                          verbose_eval=50
                          )
          
    mean_merror = pd.Series(cv_results['auc-mean']).min()
    boost_rounds = pd.Series(cv_results['auc-mean']).argmin()
    print('------------------------------------------------')    
    print('Leaves:',num_leaves)
    print('max_depth:',max_depth)
    print('BestAUC:',mean_merror)
    print('------------------------------------------------')    
    if mean_merror > min_merror:
        min_merror = mean_merror
        best_params['num_leaves'] = num_leaves
        best_params['max_depth'] = max_depth
          
params['num_leaves'] = best_params['num_leaves']
params['max_depth'] = best_params['max_depth']

设置参数
交叉验证
调参1：提高准确率
[50]	cv_agg's auc: 0.987275 + 0.00131945
[100]	cv_agg's auc: 0.987622 + 0.00101179
[150]	cv_agg's auc: 0.987833 + 0.000908506
[200]	cv_agg's auc: 0.987961 + 0.000780898
[250]	cv_agg's auc: 0.987875 + 0.000701515
------------------------------------------------
Leaves: 50
max_depth: 13
BestAUC: 0.9649348425595282
------------------------------------------------
[50]	cv_agg's auc: 0.987053 + 0.00129793
[100]	cv_agg's auc: 0.987901 + 0.000885383
[150]	cv_agg's auc: 0.988142 + 0.000826954
------------------------------------------------
Leaves: 60
max_depth: 13
BestAUC: 0.9668108334899546
------------------------------------------------
[50]	cv_agg's auc: 0.987264 + 0.00123104
[100]	cv_agg's auc: 0.987829 + 0.00106227
[150]	cv_agg's auc: 0.987966 + 0.00109905
[200]	cv_agg's auc: 0.988006 + 0.00114095
------------------------------------------------
Leaves: 70
max_depth: 13
BestAUC: 0.9671516728953722
------------------------------------------------
[50]	cv_agg's auc: 0.

KeyboardInterrupt: 